# Praca domowa 2
### Realizacja poleceń SQL w pakiecie pandas

Rozwiązanie każdego zadania składa się się z 5 podpunktów:

1. Słowna interpretacja zapytania
2. Zapytanie w SQL
3. Zapytanie w pandas
4. Porównanie wyników
5. Porównianie czasów wykonań zapytań

Zaimportowanie niezbędnych bibliotek:

In [1]:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3
import tempfile
from timeit import timeit

Załadowanie zbiorów danych:

In [2]:
Badges = pd.read_csv("travel_stackexchange_com/Badges.csv.gz", compression = 'gzip')
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv.gz", compression = 'gzip')
PostLinks = pd.read_csv("travel_stackexchange_com/PostLinks.csv.gz", compression = 'gzip')
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv.gz", compression = 'gzip')
Tags = pd.read_csv("travel_stackexchange_com/Tags.csv.gz", compression = 'gzip')
Users = pd.read_csv("travel_stackexchange_com/Users.csv.gz", compression = 'gzip')
Votes = pd.read_csv("travel_stackexchange_com/Votes.csv.gz", compression = 'gzip')

W celu sprawdzania wykonywanych poleceń, ramki danych są eksportowane do bazy danych SQLite:

In [3]:
# sciezka dostępu do bazy danych
baza = os.path.join(tempfile.mkdtemp(), 'baza.db')
# połączenie do bazy danych
conn = sqlite3.connect(baza) 
# eksportowanie ramek danych do bazy
Badges.to_sql("Badges", conn) 
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

## Zapytanie 1

```sql
SELECT Posts.Title, RelatedTab.NumLinks
FROM
    (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
    FROM PostLinks
    GROUP BY RelatedPostId) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC
```

### 1.1 Słowna interpretacja zapytania

<!---
Pogrupuj elementy z tabeli *PostLinks* po polu *RelatedPostId*. Dla każdej grupy zwróć wartość *PostId* która oznacza *RelatedPostId* dla danej grupy oraz ilość elementów grupy jako *NumLinks*. Otrzymaną tabelę oznacz jako *RelatedTab*. Następnie połącz ją z tabelą *Posts* względem pól *PostId* i *Id*. 
-->

Należy wypisać tytuły postów z tabeli *Posts* o typie 1 i dla każdego wypisać ilość postów połączonych z nim poprzez zliczenie połączeń w tabeli *PostLinks*. 

### 1.2 Zapytanie w SQL

Do wywoływania metody *read_sql_query* z pakietu pandas zdefiniujemy pomocniczą funkcję *get_sql_dataframe*, która przujmie zapytanie SQL i zwróci wynikową ramkę danych:

In [4]:
def get_sql_dataframe(query):
    return pd.read_sql_query(query, conn)

In [5]:
query1 = """
SELECT Posts.Title, RelatedTab.NumLinks
FROM
    (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
    FROM PostLinks
    GROUP BY RelatedPostId) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC
"""

In [6]:
sql_result1 = get_sql_dataframe(query1)

### 1.3 Zapytanie w pandas

In [7]:
def get_pandas_dataframe_1():
    """Funkcja realizująca zapytanie 1."""
    # Pogrupowanie elementów PostLinks i obliczenie ich ilość w każdej podgrupie, ustawienie nazw kolumn 
    RelatedTab = pd.DataFrame(PostLinks.groupby(["RelatedPostId"]).size(), columns=["NumLinks"]).reset_index().rename(columns={"RelatedPostId": "PostId"})
    # Połączenie tabeli RelatedTab z postami o typie==1 
    join = RelatedTab.merge(Posts.loc[Posts['PostTypeId'] == 1,:], how="inner", left_on="PostId", right_on="Id")
    # Wybranie odpowiednich kolumn i posortowanie wyniku malejąco, reset indeksu
    pd_result = join[["Title", "NumLinks"]].sort_values(by=["NumLinks"], ascending=False).reset_index(drop=True)
    return pd_result

# Wowłanie funkcji
pd_result1 = get_pandas_dataframe_1()

### 1.4 Porównanie wyników

In [8]:
sql_result1.equals(pd_result1)

False

Pierwsze porównianie wyników zwraca wartość False, zatem zobaczmy jak wyglądają te dwie ramki danych:

In [9]:
sql_result1

,Title,NumLinks
0,Is there a way to find out if I need a transit...,594
1,Do I need a visa to transit (or layover) in th...,585
2,Should my first trip be to the country which i...,331
3,Should I submit bank statements when applying ...,259
4,How much electronics and other valuables can I...,197
...,...,...
4867,Is Cairns the only direct destination from the...,1
4868,EasyJet - Is the return flight valid if I miss...,1
4869,Are CBP officers allowed to search and clone m...,1
4870,US B1/B2 VISA not approved,1


In [10]:
pd_result1

,Title,NumLinks
0,Is there a way to find out if I need a transit...,594
1,Do I need a visa to transit (or layover) in th...,585
2,Should my first trip be to the country which i...,331
3,Should I submit bank statements when applying ...,259
4,How much electronics and other valuables can I...,197
...,...,...
4867,Good GPS device to tracking my travels?,1
4868,Why are flights with hotel bookings cheaper?,1
4869,Riding on a narrowboat in London,1
4870,How to get a UK visa while also travelling as ...,1


Jak widać, początkowe wartości wyników zapytań są takie same, ale dla takiej samej wartości NumLinks wiersze są w innej kolejności. 

In [11]:
# Porównianie czy unikalne wartości w kolumnach NumLinks są sobie równe
(pd_result1["NumLinks"].unique() == sql_result1["NumLinks"].unique()).all()

True

In [12]:
# Porównanie czy ramki danych mają tą samą liczbę wierszy
pd_result1.shape[0] == sql_result1.shape[0]

True

Z powyższych równości wynika, że wyniki mają taką samą liczbę wierszy oraz mają te same wartości w kolumnie NumLinks.
Aby porównać ramki danych z dokładnością do różnej kolejności wierszy w danej podgrupie zdefiniujmy pomocniczą funkcję **check_equals_rows_disorder**:

In [13]:
def check_equals_rows_disorder(df_sql, df2_pd, column_name):
    """
    Funkcja sprawdza czy dwie ramki danych są równe z dokładnością do kolejności wierszy 
    względem wartości w kolumnie column_name.
    """
    # Sprawdzenie czy kolumny zawierają te same unikalne wartości w tej samej kolejności
    if not (df_sql[column_name].unique() == df2_pd[column_name].unique()).all():
        print("Unikalne warotści kolumny {0} nie są równe".format(column_name))
        return False
    
    # Sprawdzenie czy ramki danych mają tą samą liczbę wierszy
    if not (df_sql.shape[0] == df2_pd.shape[0]):
        print("Podane ramki danych mają różną liczbę wierszy")
        return False
    
    # Przejście po każdej unikalnej wartości z kolumny
    for param in df2_pd[column_name].unique():
        # Wybranie podzbiorów z ramek dla danej wartości
        df1 = df_sql.loc[df_sql[column_name] == param,:]
        df2 = df2_pd.loc[df2_pd[column_name] == param,:]  
        
        # Posortowanie wartości w podramce po wszystkich kolumnach, zresetowanie indeksów
        df11 = df1.sort_values(by=df1.columns.tolist()).reset_index(drop=True)
        df22 = df2.sort_values(by=df2.columns.tolist()).reset_index(drop=True)
        
        # Porównanie wybranych fragmentów ramek danych
        if not df11.equals(df22):
            print("Wiersze nie są równe dla {0} = {1}".format(column_name, param))
            return False
        
    return True

Następnie wywołajmy funkcję na porównywanych ramkach danych:

In [14]:
check_equals_rows_disorder(pd_result1, sql_result1, "NumLinks")

True

Zwrócona wartość True oznacza, że ramka pd_result1 jest równa ramce sql_result1. Zatem wynik jest poprawny. 

### 1.5 Porównianie czasów wykonań zapytań

Średni czas wykonania zapytania SQL dla 5 prób:

In [15]:
timeit(lambda: get_sql_dataframe(query1), number=5)

0.27830590000002076

Średni czas wykonania zapytania za pomocą funkcji z pandas dla 5 prób:

In [16]:
timeit(lambda: get_pandas_dataframe_1(), number=5)

0.2605282000000102

Widzimy, że średni czas wykonania zapytań jest lepszy dla funkcji z pakietu pandas.

# Zapytanie 2

```sql
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10

```

### 2.1 Słowna interpretacja zapytania

<!---
Z tabeli *Posts* wybierz tylko elementy o *PostTypeId* równym 1. Następnie pogrupuj elementy po id właściciela, czyli polu *OwnerUserId*. Dla każdej grupy oblicz sumę oraz maksymalną wartość *FavoriteCount*, oznacz je jako *FavoriteTotal* oraz *MostFavoriteQuestionLikes*. Dla wybranych postów dołącz tabelę *Users* tak aby id użytkownika zgadzało się z id właściciela posta. Z tabeli *Users* wybierz pola: *DisplayName*, *Age*, *Location*, a z tabeli *Posts* wybierz tytuł (ze zmienioną nazwą kolumny na *MostFavoriteQuestion*) oraz obliczone wcześniej kolumny *FavoriteTotal* oraz *MostFavoriteQuestionLikes*. Wyniki posortuj malejąco po polu *FavoriteTotal* i zwróć 10 pierwszych wyników. 
-->

Należy stworzyć ranking 10 użytkowników którzy mają największą sumę *FavoriteCount* z postów których są właścicielami. Dla każdego takiego użytkownika należy wypisać jego wyświatlaną nazwę (*DisplayName*), wiek (*Age*), lokalizację (*Location*), obliczoną sumę *FavoriteCount* (jako *FavoriteTotal*), tytuł jego posta który miał największą wartość *FavoriteCount* (jako *MostFavoriteQuestion*) oraz ilość *FavoriteCount* dla tego posta (jako *MostFavoriteQuestionLikes*). 


### 2.2 Zapytanie w SQL

In [17]:
query2 = """
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
"""

In [18]:
sql_result2 = get_sql_dataframe(query2)

### 2.3 Zapytanie w pandas

In [19]:
def get_pandas_dataframe_2():
    """Funkcja realizująca zapytanie 2."""
    # Wybranie postów o typie==1
    posts_type1 = Posts.loc[Posts["PostTypeId"] == 1, :]
    # Pogrupowanie postów po id właściciela i obliczenie sumy oraz maksimum od FavoriteCount
    aggr = posts_type1.groupby(["OwnerUserId"])['FavoriteCount'].agg([('FavoriteTotal', sum), ('MostFavoriteQuestionLikes', max)])
    # Wybranie z postów potrzebnych kolumn
    post_titles = posts_type1[["OwnerUserId", "Title", "FavoriteCount"]]
    # Połaczenie obliczonych wartości sum i max dla postów z  post_titles w celu dopasowania tytułów do najlepszych postów
    join1 = post_titles.merge(aggr, left_on=["OwnerUserId", "FavoriteCount"], right_on=["OwnerUserId","MostFavoriteQuestionLikes"])
    # Połączenie postów z użytkownikami, którzy są ich właścicielami, reset indeksowania
    join2 = join1.merge(Users[["Id", "DisplayName", "Age", "Location"]], left_on="OwnerUserId", right_on="Id").reset_index()
    # Zmiana nazw kolumn i wybranie kolumn w odpowiedniej kolejności
    pd_result = join2.rename(columns={'Title': "MostFavoriteQuestion"})[["DisplayName", "Age", "Location", "FavoriteTotal", "MostFavoriteQuestion", "MostFavoriteQuestionLikes"]]   
    # Posortowanie wierszy malejąco po wartości FavoriteTotal i wybranie 10 pierwszych wyników
    pd_result = pd_result.sort_values(by=["FavoriteTotal"], ascending=False).head(10)
    
    return pd_result.reset_index(drop=True)

# Wowłanie funkcji
pd_result2 = get_pandas_dataframe_2()

Otrzymana ramka danych funkcją get_pandas_dataframe_2():

In [20]:
pd_result2

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,NaN,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,NaN,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,NaN,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


### 2.4 Porównanie wyników

In [21]:
pd_result2.equals(sql_result2)

True

Porównianie za pomocą funkcji *equals* zwróciło True, zatem otrzymane ramki danych są równe.  

### 2.5 Porównianie czasów wykonań zapytań

Średni czas wykonania zapytania SQL dla 5 prób:

In [22]:
timeit(lambda: get_sql_dataframe(query2), number=5)

0.6439022000000136

Średni czas wykonania zapytania za pomocą funkcji pandas dla 5 prób:

In [23]:
timeit(lambda: get_pandas_dataframe_2(), number=5)

0.45353990000000977

W tym przypdadku również funkcja wykorzystująca metody z pakiety pandas ma lepszy średni czas wykonania.

# Zapytanie 3

```sql
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
    SELECT
        PostID,
        UserID,
        SUM(Score) AS CommentsTotalScore
    FROM Comments
    GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
```

### 3.1 Słowna interpretacja zapytania

Stworzenie rankingu 10 postów typu 1, które mają największą zsumowaną wartość *Score* dla komentarzy których autorem jest autor posta. Dla każdego z tych postów należy wypisać jego tytułu (*Title*) oraz zsumowaną liczbę *Score* (jako *CommentsTotalScore*). 

### 3.2 Zapytanie w SQL

In [24]:
query3 = """
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
    SELECT
        PostID,
        UserID,
        SUM(Score) AS CommentsTotalScore
    FROM Comments
    GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10

"""

In [25]:
sql_result3 = get_sql_dataframe(query3)

### 3.3 Zapytanie w pandas

In [26]:
def get_pandas_dataframe_3():
    """Funkcja realizująca zapytanie 3."""
    # Pogrupowanie komentarzy z Comments po PostId i UserId, obliczenie sumy po Score jako CommentsTotalScore
    CmtTotScr = Comments.groupby(["PostId", "UserId"])["Score"].agg([('CommentsTotalScore', sum)]).reset_index()
    # Wybranie postów typu 1
    posts_type1 = Posts.loc[Posts["PostTypeId"] == 1, :]
    # Połączenie komentarzy z postami po id postów i id userów 
    join = CmtTotScr.merge(posts_type1, left_on=["PostId", "UserId"], right_on=["Id", "OwnerUserId"])
    # Wybranie odpowiednich kolumn i posortowanie wyniku malejąco po CommentsTotalScore, wybranie 10 pierwszych wierszy
    pd_result = join[["Title", "CommentsTotalScore"]].sort_values(by=["CommentsTotalScore"], ascending=False).head(10)
    return pd_result.reset_index(drop=True)

# Wynik funkcji
pd_result3 = get_pandas_dataframe_3()

### 3.4 Porównanie wyników

In [27]:
sql_result3.equals(pd_result3)

False

Pierwsze porównanie ramek danych zwraca False, zobaczmy więc jak one wyglądają:

In [28]:
pd_result3

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,Can I have a watermelon in hand luggage?,25
4,How does President Trump's travel ban affect n...,25
5,What's the longest scheduled public bus ride i...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,India just demonetized all Rs 500 & 1000 notes...,20
9,Legalities and safety concerns of visiting pro...,20


In [29]:
sql_result3

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


Łatwo zauważyć, że również jest to tylko problem permutacji wierszy w obrębie tej samej wartości *CommentsTotalScore*, zatem możemy znowu użyć funkcji *check_equals_rows_disorder*:

In [30]:
check_equals_rows_disorder(pd_result3, sql_result3, "CommentsTotalScore")

True

Zwrócona wartość jest True, zatem zdefiniowana funkcja wywołująca metody z pakietu pandas jest poprawna.

### 3.5 Porównianie czasów wykonań zapytań

Średni czas wykonania zapytania SQL dla 5 prób:

In [31]:
timeit(lambda: get_sql_dataframe(query3), number=5)

1.957180200000039

Średni czas wykonania zapytania za pomocą funkcji pandas dla 5 prób:

In [32]:
timeit(lambda: get_pandas_dataframe_3(), number=5)

0.9705718999999817

Średni czas wykonania zapytnia poprzez użycie metod z pakietu pandas jest dwukrotnie krótszy niż wywołanie *read_sql_query* dla tego przypadku. 

# Zapytanie 4

```sql
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
    SELECT
        Name, UserID
    FROM Badges
    WHERE Name IN (
        SELECT
        Name
        FROM Badges
        WHERE Class=1
        GROUP BY Name
        HAVING COUNT(*) BETWEEN 2 AND 10
    )
    AND Class=1
    ) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id
```

### 4.1 Słowna interpretacja zapytania

Wybranie użytkowników, którzy mają przypisane odznaki (*Badges*) spełniające następujące kryteria: są klasy 1 i mają pomiędzy 2 a 10 elementów w obrębie danej nazwy. Należy wypsać id użytkownika (*Id*), jego wyświetlaną nazwę (*DisplayName*), reputację (*Reputation*), wiek (*Age*) oraz lokalizację (*Location*).

### 4.2 Zapytanie w SQL

In [33]:
query4 = """
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
    SELECT
        Name, UserID
    FROM Badges
    WHERE Name IN (
        SELECT
        Name
        FROM Badges
        WHERE Class=1
        GROUP BY Name
        HAVING COUNT(*) BETWEEN 2 AND 10
    )
    AND Class=1
    ) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id
"""

In [34]:
sql_result4 = get_sql_dataframe(query4)

### 4.3 Zapytanie w pandas

In [35]:
def get_pandas_dataframe_4():
    """Funkcja realizująca zapytnie 4."""
    # Wybranie odznak klasy 1 z Badges
    badges_class1 = Badges[Badges["Class"]==1].reset_index()
    # Pogrupowanie odznak po nazwie i obliczenie ich ilości w każdej podgrupie
    badges_names = pd.DataFrame(badges_class1.groupby("Name").size(), columns=["COUNT(*)"]).reset_index()
    # Wybranie odznak których ilość jest pomiędzy 2 a 10
    badges_names = badges_names[(badges_names["COUNT(*)"] >= 2) & (badges_names["COUNT(*)"] <= 10)]
    # Wybranie wierszy z badges_class1, których nazwa odznaki znajduje się w badges_names
    ValuableBadges = badges_class1[badges_class1["Name"].isin(badges_names["Name"])][["Name", "UserId"]].reset_index(drop=True)
    # Wybranie potrzebnych kolumn z tabeli Users
    users = Users[["Id", "DisplayName", "Reputation", "Age", "Location"]]
    # Połączenie ValuableBadges z Users po id użytkownika
    join = ValuableBadges.merge(users, left_on="UserId", right_on="Id")
    # Wybranie odpowiednich kolumn, usunięcie duplikatów
    pd_result = join[["Id", "DisplayName", "Reputation", "Age", "Location"]].drop_duplicates()
    return pd_result.reset_index(drop=True)

#Wynik funkcji
pd_result4 = get_pandas_dataframe_4()

### 4.4 Porównanie wyników

In [36]:
pd_result4.equals(sql_result4)

True

Otrzymane ramki danych są równe.

### 4.5 Porównianie czasów wykonań zapytań

Średni czas wykonania zapytania SQL dla 5 prób:

In [37]:
timeit(lambda: get_sql_dataframe(query4), number=5)

0.5195544999999697

Średni czas wykonania zapytania za pomocą funkcji pandas dla 5 prób:

In [38]:
timeit(lambda: get_pandas_dataframe_4(), number=5)

0.1560077999999976

Średni czas wykonania zapytania funkcją *get_pandas_dataframe_4* jest dużo krótszy niż wykonanie zapytania SQL.

# Zapytanie 5

```sql
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
    ) AS BestAnswers
JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
    ) AS Questions
    ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC

```

### 5.1 Słowna interpretacja zapytania

Wypisanie pytań dla których różnica pomiędzy wynikiem (*Score*) dla najlepszej odpowiedzi a zaakceptowanej odpowiedzi jest większa niż 50. Należy posortować malejąco pytania po tej różnicy i wypisać id pytania (*Id*), tytuł pytania (*Title*), maksymalny wynik dla udzielonych odpowiedzi do tego pytania (*MaxScore*), wynik zaakceptowanej odpowiedzi dla tego pytania (*AcceptedScore*) oraz różnicę pomiędzy dwoma poprzednimi (*Difference*). 

Pytania to posty typu 1, a odpowiedzi to posty typu 2. 

### 5.2 Zapytanie w SQL

In [39]:
query5 = """
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
    ) AS BestAnswers
JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
    ) AS Questions
    ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
"""

In [40]:
sql_result5 = get_sql_dataframe(query5)

### 5.3 Zapytanie w pandas

In [41]:
def get_pandas_dataframe_5():
    """Funkcja realizująca zapytnie 4."""
    # Wybranie postów o typie 2 czyli odpowiedzi
    answers = Posts[Posts["PostTypeId"]==2]
    # Pogrupowanie odpowiedzi po ParentId i obliczenie maksimum z Score
    BestAnswers = answers.groupby("ParentId")["Score"].agg([('MaxScore', max)])
    # Połączenie BestAnswers z answers w celu dołączenia Id wybranych odpowiedzi
    BestAnswers = BestAnswers.merge(answers, left_on=["ParentId", "MaxScore"], right_on=["ParentId", "Score"])
    # Wybranie odpowiednich kolumn
    BestAnswers = BestAnswers[["Id", "ParentId", "MaxScore"]].reset_index(drop=True)
    # Wybranie postów o typie 1 czyli pytań
    Questions = Posts[Posts["PostTypeId"]==1]
    # Połączenie wybranych odpowiedzi z pytaniami
    join1 = BestAnswers.merge(Questions, left_on="ParentId", right_on="Id", suffixes=("_ans", "_que"))
    # Połączenie pytań i odpowiedzi z postami na podstawie id zaakceptowanej odpowiedzi
    join2 = join1.merge(Posts, left_on="AcceptedAnswerId", right_on="Id", suffixes=("_qa", "_posts"))
    # Obliczenie różnicy pomiędzy maksymalnym wynikiem, a wynikiem zaakceptowanej odpowiedzi
    join2["Difference"] = join2["MaxScore"] - join2["Score_posts"]
    # Wybranie wierszy, dla których różnica jest większa niż 50
    pd_result = join2[join2["Difference"]>50]
    # Wybranie odpowiednich kolumn i zmienienie odpowiednich nazw
    pd_result = pd_result[["Id_que", "Title_qa", "MaxScore", "Score_posts", "Difference"]].rename(columns={
        "Id_que":"Id", 
        "Title_qa":"Title", 
        "Score_posts":"AcceptedScore"
    })
    # Posortowanie wierszy malejąco po Difference
    pd_result = pd_result.sort_values(by=["Difference"], ascending=False)
    return pd_result.reset_index(drop=True)

# Wynik funkcji
pd_result5 = get_pandas_dataframe_5()

### 5.4 Porównanie wyników

In [42]:
sql_result5.equals(pd_result5)

True

Z powyższego wywołania funkcji *equals* wynika, że uzyskane ramki danych są sobie równe, czyli funkcja *get_pandas_dataframe_5* jest poprawna. 

Wynikowa ramka danych składa się tylko z 8 wierszy, więc możemy ją w całości wyświetlić: 

In [43]:
pd_result5

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


### 5.5 Porównianie czasów wykonań zapytań

Średni czas wykonania zapytania SQL dla 5 prób:

In [44]:
timeit(lambda: get_sql_dataframe(query5), number=5)

1.176358300000004

Średni czas wykonania zapytania za pomocą funkcji pandas dla 5 prób:

In [45]:
timeit(lambda: get_pandas_dataframe_5(), number=5)

1.7329952999999705

W ostatnim przypadku średni czas wykonania zapytania za pomocą funkcji z pakietu pandas jest dłuższy niż wywołanie SQL. 

## Podsumowanie

Wszystkie zaimplementowane funkcje wykorzystujące metody z pakietu pandas zwracają poprawne ramki danych. W 4 na 5 przypadków ich średnie czasy wykonania są krótsze niż wywołanie metody read_sql_query dla danego zapytania SQL. 

Na koniec pozostaje zamknąć połączenie do wykorzystywanej bazy danych:

In [46]:
# zamknięcie połączenia do bazy danych
conn.close()